# TIN Geometry Parsing and Centroid Calculation

In [1]:
# -------------------------------------------------------------------------
# Author: Farid Javadnejad
# Date: 2025-02-05
# Description: 
# This script parses a TIN geometry from a LandXML file, computes triangle areas,
# and calculates the area-weighted centroid of the TIN.
# Functions:
#   - parse_tin_geometry: Parses the XML file to extract points and faces.
#   - compute_triangle_area: Computes the area of a triangle using the cross-product method.
#   - compute_weighted_centroid: Computes the area-weighted centroid of a TIN.
# Disclaimer: 
# AI-assisted debugging, reviewing and testing were used in this script.
# -------------------------------------------------------------------------

## Libraries

In [2]:
from bs4 import BeautifulSoup as bs
import numpy as np

## Helper Functions

In [10]:
def parse_tin_geometry(xml_file):
    """
    Parses a TIN geometry from a LandXML file and returns points and faces.

    Steps:
    1. Reads the XML file using BeautifulSoup.
    2. Extracts faces from <F> tags, skipping those with i="1"
       read: <F n="5 2 6">9 5 6</F>
       return: [[9, 5, 6]]
    3. Extracts points from <P> tags.
       read: <P id="100">1.0 2.0 3.0</P>
       return: {100: np.array([1.0, 2.0, 3.0])}
    """
    points = {}
    faces = []

    # Read the XML file & catch any exception
    try:
        with open(xml_file, 'r', encoding='utf-8') as file:
            xml_soup = bs(file, 'lxml-xml')
        print (f"Reading {xml_file}")
    except Exception as e:
        raise ValueError(f"Error reading XML file: {e}")


    # Extract points
    print("\nExtracting points...")
    counter = 0
    for point_tag in xml_soup.find_all('P'):
        pid = int(point_tag["id"])
        point_list = []
        for x in point_tag.text.split():
            point_list.append(float(x))
        points[pid] = np.array(point_list)
        
        # Count number of points
        counter += 1
    print(f"Number of points: {counter}")


    # Extract faces
    print("\nExtracting faces...")
    counter = 0
    for face in xml_soup.find_all('F'):
        if face.get("i") == "1":
            continue  # Skip flagged faces
        face_list = []
        for x in face.text.split():
            face_list.append(int(x))
        faces.append(face_list)

        # Count number of points
        counter += 1
    print(f"Number of faces: {counter}")

    # Return points and faces
    return points, faces



def compute_triangle_area(p1, p2, p3):
    """
    Computes the area of a triangle in 3D space using the cross-product method.
    Area = 0.5 * ||v1 x v2|| = 0.5 ||(p2-p1) x (p3-p1)||
    """
    # Compute the vectors from point p1
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate the cross product
    cross_product = np.cross(v1, v2)

    # Compute the area using the magnitude of the cross product
    area = 0.5 * np.linalg.norm(cross_product)
    return area



def compute_weighted_centroid(points, faces):
    """ 
    Computes the area-weighted centroid of a Triangulated Irregular Network (TIN).
    1. Compute the centroid of each triangle
        Ci = (p1 + p2 + p3) / 3
    2. Compute the area of each triangle
        Ai = 0.5 * ||(p2-p1) x (p3-p1)||
    3. Compute the total weighted centroid
        Ctin = Σ(Ai * Ci) / Σ(Ai)
    """
    total_weighted_centroid = np.zeros(3)
    total_area = 0
    
    print_counter = 0

    print("\nCalculating centroids and areas...")
    for face in faces:
        p1, p2, p3 = points[face[0]], points[face[1]], points[face[2]]
        area = compute_triangle_area(p1, p2, p3)     
        centroid = (p1 + p2 + p3) / 3.0
        total_weighted_centroid += area * centroid
        total_area += area

    print(f"Total Area:{total_area}")
    weighted_centroid = total_weighted_centroid / total_area if total_area else None
    
    return weighted_centroid

## Main

In [11]:
# Set file address (Note: Use "/" OR "\\" instead of "\" for addressing the file path)
xml_file = 'C:\\Farid\\gitProjects\\tin_geometry_parsing_and_calculations\\notebook\\sample_xml.xml'
#xml_file1 = 'C:\\Farid\\gitProjects\\tin_geometry_parsing_and_calculations\\data\\raw\\NM830224_XDTM_USGS.xml'

# Load XML and parse data
points, faces = parse_tin_geometry(xml_file)

# Compute area-weighted centroid
weighted_centroid = compute_weighted_centroid(points, faces)
print("Area-weighted centroid:", weighted_centroid)


Reading C:\Farid\gitProjects\tin_geometry_parsing_and_calculations\notebook\sample_xml.xml

Extracting points...
Number of points: 5

Extracting faces...
Number of faces: 3

Calculating centroids and areas...
Total Area:43.80082769512965
Area-weighted centroid: [3.94278026 3.50221894 0.89427803]
